In [ ]:
import os

# Đường dẫn đến thư mục cần duyệt
folder_path = 'labels/train/'

def convert_to_yolo_format(data):
    yolo_data = []
    for line in data:
        
        values = list(map(float, line.strip().split()))
        if(len(values) == 9):
            class_id = int(values[0])
            x1, y1, x2, y2, x3, y3, x4, y4 = values[1:]

            x_min = min(x1, x2, x3, x4)
            x_max = max(x1, x2, x3, x4)
            y_min = min(y1, y2, y3, y4)
            y_max = max(y1, y2, y3, y4)

            # Trung bình x và y của các điểm
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2

            # Kích thước width và height
            width = x_max - x_min
            height = y_max - y_min

            # Thêm vào danh sách định dạng YOLO
            yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")
        else:
            print("Invalid data: ", values)
            continue
    return yolo_data

# Duyệt qua tất cả các file trong thư mục
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Kiểm tra nếu là file và có đuôi .txt
    if os.path.isfile(file_path) and filename.endswith('.txt'):  
        with open(file_path, 'r') as file:
            data = file.readlines()
        yolo_results = convert_to_yolo_format(data)
        # Ghi đè nội dung mới vào file (xóa toàn bộ nội dung cũ)
        with open(file_path, 'w') as file:
            for result in yolo_results:
                file.write(result + "\n")  # Ghi trực tiếp chuỗi vào file

print("Done!")


In [5]:
import os
import cv2
import numpy as np

# Đường dẫn đến thư mục ảnh và nhãn
image_folder_path = 'yoloData\\OCR\\images\\train'
label_folder_path = 'yoloData\\OCR\\labels\\train'
output_folder_path = 'YoloImage'  # Thư mục để lưu ảnh với bounding box
# label_output_folder_path = 'YoloLabel/'

# Tạo thư mục đầu ra nếu chưa có
os.makedirs(output_folder_path, exist_ok=True)

In ra bounding box với giá trị ban đầu

In [1]:
import cv2
import os
import numpy as np

def printBoundingBox(image_filename):
    # Tạo tên file nhãn tương ứng
    label_filename = image_filename.replace('.jpg', '.txt')

    image_file_path = os.path.join(image_folder_path, image_filename)
    label_file_path = os.path.join(label_folder_path, label_filename)

    # Kiểm tra nếu cả file ảnh và nhãn đều tồn tại
    if os.path.isfile(image_file_path) and os.path.isfile(label_file_path):
        # Đọc ảnh
        image = cv2.imread(image_file_path)

        # Kiểm tra xem ảnh có được đọc thành công không
        if image is None:
            print(f"Không thể mở ảnh: {image_file_path}")
            return

        # Đọc file nhãn
        with open(label_file_path, 'r') as file:
            data = file.readlines()

        # Xử lý từng dòng trong file nhãn
        for line in data:
            # Chuyển chuỗi thành danh sách các số
            values = list(map(float, line.split()))
            
            # Lấy giá trị đầu tiên và các tọa độ 4 điểm còn lại
            if len(values) == 9:
                label_value = values[0]  # Lấy giá trị đầu tiên
                coordinates = values[1:]  # Lấy các tọa độ
                
            else:
                print(f"Số lượng giá trị không đúng trong file {label_filename}: {values}")
                continue

            # Lấy các tọa độ 4 điểm từ file nhãn
            x1, y1, x2, y2, x3, y3, x4, y4 = coordinates

            # Nhân tất cả giá trị với kích thước ảnh
            x = [x1, x2, x3, x4]
            y = [y1, y2, y3, y4]

            x = [coord * image.shape[1] for coord in x]
            y = [coord * image.shape[0] for coord in y]

            x1, x2, x3, x4 = x
            y1, y2, y3, y4 = y

            # Vẽ bounding box bằng cách nối các điểm
            points = np.array([[int(x1), int(y1)],
                               [int(x2), int(y2)],
                               [int(x3), int(y3)],
                               [int(x4), int(y4)],
                               [int(x1), int(y1)]], np.int32)
            points = points.reshape((-1, 1, 2))

            # Vẽ bounding box trên ảnh
            cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)  # Bounding box màu xanh lá
            
            # Hiển thị giá trị đầu tiên bên cạnh bounding box
            text_position = (int(x1), int(y1) - 10)  # Đặt vị trí văn bản phía trên góc trên bên trái của bounding box
            cv2.putText(image, str(int(label_value)), text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)  # Văn bản màu đỏ

        # Lưu ảnh với bounding box vào thư mục đầu ra
        output_file_path = os.path.join(output_folder_path, image_filename)
        cv2.imwrite(output_file_path, image)


In [ ]:
# Duyệt qua tất cả các file trong thư mục ảnh
for image_filename in os.listdir(image_folder_path):
    if image_filename.endswith('.jpg'):
        printBoundingBox(image_filename)

print("Bounding boxes drawn and images saved!")

In ra bounding box để check yolo data is correct

In [6]:
def convert_from_yolo_to_points(values):
    class_id, x_center, y_center, width, height = values
    
    # Chuyển đổi từ YOLO về tọa độ góc trái trên và góc phải dưới
    x1 = x_center - width / 2
    y1 = y_center - height / 2
    x2 = x_center + width / 2
    y2 = y_center - height / 2
    x3 = x_center + width / 2
    y3 = y_center + height / 2
    x4 = x_center - width / 2
    y4 = y_center + height / 2
    return [class_id, x1, y1, x2, y2, x3, y3, x4, y4]

In [ ]:
# Duyệt qua tất cả các file trong thư mục ảnh
for image_filename in os.listdir(image_folder_path):
    if image_filename.endswith('.jpg'):
        label_filename = image_filename.replace(".jpg", ".txt")
        
        image_file_path = os.path.join(image_folder_path, image_filename)
        label_file_path = os.path.join(label_folder_path, label_filename)

        if os.path.isfile(image_file_path) and os.path.isfile(label_file_path):
            image = cv2.imread(image_file_path)

            # Kiểm tra đọc ảnh có thành công hay không
            if image is None:
                print(f"Không thể mở ảnh: {image_file_path}")
                continue

            # Đọc file nhãn
            with open(label_file_path, 'r') as file:
                data = file.readlines()
            
            # Xử lý từng dòng
            for line in data:
                values = list(map(float, line.strip().split()))

                if len(values) == 5:
                    values = convert_from_yolo_to_points(values)
                else:
                    print(f"Số lượng giá trị không đúng trong file {label_filename}: {values}")
                    continue

                # Lấy các tọa độ 4 điểm từ file nhãn
                label_value, x1, y1, x2, y2, x3, y3, x4, y4 = values

                x = [x1, x2, x3, x4]
                y = [y1, y2, y3, y4]

                x = [coord * image.shape[1] for coord in x]
                y = [coord * image.shape[0] for coord in y]

                x1, x2, x3, x4 = x
                y1, y2, y3, y4 = y

                # Vẽ bounding box bằng cách nối các điểm
                points = np.array([[int(x1), int(y1)],
                                [int(x2), int(y2)],
                                [int(x3), int(y3)],
                                [int(x4), int(y4)],
                                [int(x1), int(y1)]], np.int32)
                points = points.reshape((-1, 1, 2))

                # Vẽ bounding box trên ảnh
                cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)  # Bounding box màu xanh lá
                
                # Hiển thị giá trị đầu tiên bên cạnh bounding box
                text_position = (int(x1), int(y1) - 10)  # Đặt vị trí văn bản phía trên góc trên bên trái của bounding box
                cv2.putText(image, str(int(label_value)), text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)  # Văn bản màu đỏ

            # Lưu ảnh với bounding box vào thư mục đầu ra
            output_file_path = os.path.join(output_folder_path, image_filename)
            cv2.imwrite(output_file_path, image)

print("Bounding boxes drawn and images saved!")

In [2]:
import torch

# Kiểm tra số lượng GPU
print(torch.cuda.device_count())

# Tên của GPU
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

1
NVIDIA GeForce GTX 1650


**Xoay lại ảnh để nhận thêm dữ liệu!**

In [85]:
import cv2
import numpy as np

h, w = image.shape[:2]

# Tạo tên file nhãn tương ứng

def imageRotate(image_filename):
    image_file_path = os.path.join(image_folder_path, image_filename)
    
    # Đọc ảnh
    image = cv2.imread(image_file_path)
    if image is None:
        print(f"Không thể mở ảnh")
        return
    # Lấy kích thước của ảnh
    h, w = image.shape[:2]
    # Tính toán tâm của ảnh

    # Tạo ma trận xoay
    M = cv2.getRotationMatrix2D((w/2, h/2), 315, 1.0)

    # Xoay ảnh
    rotated_image = cv2.warpAffine(image, M, (w, h))

    #Output file
    output_file_path = os.path.join(output_folder_path, "rotated45_" + image_filename)

    # Lưu ảnh đã xoay
    cv2.imwrite(output_file_path, rotated_image)

def rotate_points(points, angle, center):
    angle = np.deg2rad(angle)
    M = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])
    points = np.array(points).reshape(-1, 2)  # Chuyển đổi điểm thành ma trận
    points = points - center  # Dịch điểm về gốc tọa độ
    
    rotated_points = np.dot(points, M.T) + center  # Xoay điểm
    rotated_points = rotated_points.reshape(-1)
    
    return rotated_points.reshape(-1)

In [ ]:
# Duyệt qua tất cả các file trong thư mục ảnh
for image_filename in os.listdir(image_folder_path):
    if image_filename.endswith('.png'):
        image_file_path = os.path.join(image_folder_path, image_filename)
        label_filename = image_filename.replace('.png', '.txt')
        label_file_path = os.path.join(label_folder_path, label_filename)
        label_output = os.path.join(label_output_folder_path, "rotated45_" + label_filename)
        if os.path.isfile(label_file_path):
            imageRotate(image_filename)

            # Đọc file nhãn
            with open(label_file_path, 'r') as file:
                data = file.readlines()
            
            results_data = []
            # Xử lý từng dòng
            for line in data:
                values = list(map(float, line.strip().split()))

                if len(values) == 9:
                    class_id = values[0]
                    values = values[1:]

                    image = cv2.imread(image_file_path)

                    if image is None:
                        print(f"Không thể mở ảnh: {image_file_path}")
                        
                    for x in range(0, 8, 1):
                        if x % 2== 0:
                            values[x] *= image.shape[1]
                        else: 
                            values[x] *= image.shape[0]

                    values = rotate_points(values, 45, (image.shape[1] / 2, image.shape[0] / 2))
                    for x in range(0, 8, 1):
                        if x % 2== 0:
                            values[x] /= image.shape[1]
                        else: 
                            values[x] /= image.shape[0]

                    results_data.append(f"{class_id} {values[0]} {values[1]} {values[2]} {values[3]} {values[4]} {values[5]} {values[6]} {values[7]}")

                else:
                    print(f"Số lượng giá trị không đúng trong file {label_filename}: {values}")
                    continue

            with open(label_output, 'w') as file:
                for result in results_data:
                    file.write(result + "\n")
        
print("Completed!")

In [5]:
import cv2
import numpy as np

# Đọc hình ảnh
def motion_blur(image_path):
    image = cv2.imread(image_path)

    # Xác định kích thước của kernel (kích thước của hiệu ứng chuyển động)
    kernel_size = 15

    # Tạo kernel cho motion blur (ở đây là theo chiều ngang)
    kernel_motion_blur = np.zeros((kernel_size, kernel_size))
    kernel_motion_blur[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
    kernel_motion_blur = kernel_motion_blur / kernel_size

    # Áp dụng kernel lên hình ảnh để tạo hiệu ứng motion blur
    blurred_image = cv2.filter2D(image, -1, kernel_motion_blur)

    # Hiển thị hình ảnh gốc và hình ảnh đã bị nhiễu
    cv2.imshow('Original Image', image)
    cv2.imshow('Motion Blurred Image', blurred_image)

    # Lưu hình ảnh kết quả
    cv2.imwrite('motion_blurred_image.jpg', blurred_image)

    # Đợi người dùng nhấn phím bất kỳ rồi đóng cửa sổ hiển thị
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [3]:
import cv2
def find_best_thresholding(image_path):
    # Đọc hình ảnh màu xám
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Áp dụng Otsu's thresholding
    otsu_thresholded_image = cv2.adaptiveThreshold(image, 255, 
                                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                                   cv2.THRESH_BINARY, 
                                                   blockSize=11, 
                                                   C=2)
    # Hiển thị hình ảnh gốc và hình ảnh đã threshold bằng Otsu's method
    cv2.imshow('Original Image', image)
    cv2.imshow('Otsu Thresholded Image', otsu_thresholded_image)

    cv2.imwrite('otsu_thresholded_image.jpg', otsu_thresholded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

find_best_thresholding('1.jpg')

In [11]:
find_best_thresholding(image_path)

In [12]:
import cv2
import os

def convert_to_grayscale(image_path, save_path):
    # Đọc ảnh màu từ đường dẫn
    image = cv2.imread(image_path)

    # Chuyển đổi ảnh màu sang ảnh xám
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Lấy thư mục từ save_path
    directory = os.path.dirname(save_path)

    # Nếu thư mục không tồn tại, tạo thư mục
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Lưu ảnh xám vào đường dẫn đó
    cv2.imwrite(save_path, gray_image)

In [33]:
input_path = 'yoloData\\OCR\\images\\train'
output_path = 'newData\\gray'

In [34]:
import os
# Duyệt qua tất cả các file trong thư mục ảnh
for class_name in os.listdir(input_path):
    # Tạo đường dẫn!
    image_file_path = os.path.join(input_path, class_name)
    output_file_path = os.path.join(output_path, class_name)
    
    convert_to_grayscale(image_file_path, output_file_path)


**data augment**

In [8]:
import cv2
import numpy as np

def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
    # Step 1: Convert to grayscale (if not already)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Step 3: Sharpen the image
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(blurred, -1, kernel)

    # Step 4: Optional - Apply more advanced deblurring techniques if needed
    # Uncomment the following lines to apply Wiener filter for deblurring
    # deblurred = cv2.fastNlMeansDenoising(sharpened, None, 30, 7, 21)
    
    # Save or return the processed image
    return sharpened

# Example usage
preprocessed_image = preprocess_image("output\\bboxes\\bbox_0_label_2_conf_0.96.jpg")
cv2.imwrite("preprocessed_image.jpg", preprocessed_image)

# To show the image using OpenCV
cv2.imshow('Preprocessed Image', preprocessed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
def adjust_brightness_contrast_and_add_noise(image, brightness_range=(-30, 30), contrast_range=(0.7, 1.5)):
    """
    Adjust the brightness and contrast of an image and add random noise.
    :param image: Input image.
    :param brightness_range: Tuple (min, max) for brightness adjustment.
    :param contrast_range: Tuple (min, max) for contrast adjustment.
    :return: Adjusted and noisy image.
    """
    if image is None:
        raise ValueError("Input image is None")

    # Adjust brightness and contrast
    brightness = np.random.uniform(*brightness_range)
    contrast = np.random.uniform(*contrast_range)
    image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
    
    # Add random noise
    noise_type = np.random.choice(['gaussian', 'salt_pepper', 'poisson'])
    
    if noise_type == 'gaussian':
        row, col, ch = image.shape
        mean = 0
        var = np.random.uniform(5, 15)  # Random variance between 5 and 15
        sigma = var ** 0.5
        gauss = np.random.normal(mean, sigma, (row, col, ch))
        image = np.clip(image + gauss.astype(np.uint8), 0, 255)
    
    elif noise_type == 'salt_pepper':
        s_vs_p = np.random.uniform(0.5, 0.5)  # Ratio of salt to pepper
        amount = np.random.uniform(0.02, 0.05)  # Amount of noise between 2% and 5%
        out = np.copy(image)
        num_salt = np.ceil(amount * image.size * s_vs_p)
        num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
        
        # Add salt noise
        salt_coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape]
        for coord in zip(*salt_coords):
            out[coord] = 255
        
        # Add pepper noise
        pepper_coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape]
        for coord in zip(*pepper_coords):
            out[coord] = 0
        
        image = out
    
    elif noise_type == 'poisson':
        image = image.astype(np.float32)
        image = np.random.poisson(image).astype(np.uint8)
        image = np.clip(image, 0, 255)  # Ensure pixel values stay within [0, 255]
    
    return image

In [ ]:
import cv2
import numpy as np

def modify_image(image, action='erase'):
    """
    Modify an image by erasing a part, scaling, or shrinking with random parameters.
    :param image: Input image.
    :param action: Action to perform ('erase', 'scale', 'shrink').
    :return: Modified image.
    """
    if action == 'erase':
        erase_size = np.random.randint(30, 80)  # Random size for the erased area
        h, w = image.shape[:2]
        x = np.random.randint(0, w - erase_size)
        y = np.random.randint(0, h - erase_size)
        image[y:y + erase_size, x:x + erase_size] = 0
    
    elif action == 'scale':
        scale_factor = np.random.uniform(1.1, 1.5)  # Random scale factor between 1.1 and 1.5
        (h, w) = image.shape[:2]
        scaled = cv2.resize(image, (int(w * scale_factor), int(h * scale_factor)))
        return scaled
    
    elif action == 'shrink':
        scale_factor = np.random.uniform(0.5, 1.0)  # Random scale factor between 0.5 and 1.0
        (h, w) = image.shape[:2]
        shrunk = cv2.resize(image, (int(w / scale_factor), int(h / scale_factor)))
        return shrunk
    
    return image


In [20]:
import shutil
import os

def copy_label_file(source_path, destination_path):
    """
    Sao chép một tệp nhãn từ đường dẫn nguồn đến đường dẫn đích.
    :param source_path: Đường dẫn đến tệp nguồn.
    :param destination_path: Đường dẫn đến đích sao chép.
    """
    # Kiểm tra nếu tệp nguồn tồn tại
    if not os.path.exists(source_path):
        raise FileNotFoundError(f"Tệp nguồn không tồn tại: {source_path}")
    
    # Tạo thư mục đích nếu chưa tồn tại
    destination_directory = os.path.dirname(destination_path)
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory, exist_ok=True)
    
    # Sao chép tệp
    shutil.copy(source_path, destination_path)

In [35]:
import os
import cv2
import numpy as np

# Đường dẫn đến thư mục ảnh và nhãn
image_folder_path = 'newData\gray'
label_folder_path = 'yoloData\\OCR\\labels\\train'
output_folder_path = 'yoloData\\OCR\\images\\train'  # Thư mục để lưu ảnh với bounding box
label_output_folder_path = 'yoloData\\OCR\\labels\\train'

# Tạo thư mục đầu ra nếu chưa có
os.makedirs(output_folder_path, exist_ok=True)

In [37]:
# Duyệt qua tất cả các file trong thư mục ảnh
for image_filename in os.listdir(image_folder_path):
    if image_filename.endswith('.jpg'):
        label_filename = image_filename.replace(".jpg", ".txt")
        
        image_file_path = os.path.join(image_folder_path, image_filename)
        label_file_path = os.path.join(label_folder_path, label_filename)

        output_image_path = os.path.join(output_folder_path, "2"+image_filename)
        output_label_path = os.path.join(label_output_folder_path, "2"+label_filename)

        image = cv2.imread(image_file_path)
        image = adjust_brightness_contrast_and_add_noise(image)
        cv2.imwrite(output_image_path, image)

        copy_label_file(label_file_path, output_label_path)



Xóa các file có label không phù hợp

In [ ]:
import os

# Thay đổi đường dẫn này thành thư mục chứa các tệp .txt của bạn
directory = 'labels/train'

# Duyệt qua tất cả các tệp trong thư mục
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        
        try:
            # Đọc nội dung của tệp
            with open(file_path, 'r') as file:
                contents = file.read()
                
            # Chia nội dung thành các giá trị và chuyển thành số
            values = contents.split()
            
            # Kiểm tra các giá trị
            invalid = any(float(value) < 0 or float(value) > 1 for value in values)
            
            # Nếu có giá trị không hợp lệ, xóa tệp
            if invalid:
                os.remove(file_path)
                print(f"Đã xóa tệp: {filename}")

        except Exception as e:
            print(f"Lỗi khi xử lý tệp {filename}: {e}")
